In [1]:
import hopsworks
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import seaborn as sns
from matplotlib import pyplot
from hsml.schema import Schema
from hsml.model_schema import ModelSchema
import joblib
import os

/home/redve/Kurser Master/ID2223 Scalable Machine Learning and Deep Learning/ID2223/Lab1/Task1/threepointten/lib64/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os

for k, v in os.environ.items():
    if "HOPS" in k:
        print(f'{k}={v}')

HOPSWORKS_API_KEY=RJFCr0ixzSbcbtDM.awhoHRF08thPKMBz0x6BoX5piWGyY3DnGXfBKY3X4pCYQjxy6ypwNycQYbwPQgqj


In [7]:
# You have to set the environment variable 'HOPSWORKS_API_KEY' for login to succeed
project = hopsworks.login()
fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/190613
Connected. Call `.close()` to terminate connection gracefully.


In [16]:
wine_fg1 = fs.get_feature_group(name="winequality", version=10)
query1 = wine_fg1.select_all()
feature_view1 = fs.get_or_create_feature_view(name="winequality",
                                             version=10,
                                             description="Read from winequality feature group",
                                             labels=["quality"],
                                             query=query1)

# Read the data

In [18]:
x_train, x_test, y_train, y_test = feature_view1.train_test_split(test_size=0.2)

Finished: Reading data from Hopsworks, using ArrowFlight (1.81s) 


In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
#creating RandomForestClassifier constructor
rnd = RandomForestClassifier()
fit_rnd = rnd.fit(x_train,y_train)
rnd_score = rnd.score(x_test,y_test)
print("Random Forest Classifier Score: ",rnd_score)

Random Forest Classifier Score:  0.5778611632270169


In [ ]:

mr = project.get_model_registry()


model_dir="wine_model"
if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)

joblib.dump(model, model_dir + "/wine_model.pkl")
plt.savefig(model_dir + "/confusion_matrix.png")

input_schema = Schema(x_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema, output_schema)

wine_model = mr.python.create_model(
    name="wine_model", 
    metrics={"accuracy" : score},
    model_schema=model_schema,
    description="Wine Quality Predictor"
)


wine_model.save(model_dir)
